## Aula 14

#### Testando código genérico para compilar e executar com thrust 

In [1]:
%%writefile main.cu
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <iostream>

int main() {
    thrust::host_vector<double> host(5, 0);
    host[4] = 35;

    /* na linha abaixo os dados são copiados
       para GPU */
    thrust::device_vector<double> dev(host);    
    /* a linha abaixo só muda o vetor na CPU */
    host[2] = 12;
    
    printf("Host vector: ");
    for (auto i = host.begin(); i != host.end(); i++) {
        std::cout << *i << " "; // este acesso é rápido -- CPU
    }
    printf("\n");
    
    printf("Device vector: ");
    for (auto i = dev.begin(); i != dev.end(); i++) {
        std::cout << *i << " "; // este acesso é lento! -- GPU
    }
    printf("\n");

}

Writing main.cu


In [2]:
!nvcc -Wno-deprecated-gpu-targets -arch=sm_37 -std=c++14 -o main main.cu

In [3]:
!./main

Host vector: 0 0 12 0 35 
Device vector: 0 0 0 0 35 


#### Exercício Google Stocks

In [30]:
%%writefile google_stocks.cu

#include<iostream>
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
using namespace std;

int main() {
    thrust::host_vector<double> stocks;

    for(int i = 0;i < 2518; i++) {
        double stock;
        cin >> stock;
        stocks.push_back(stock);    
    }

    thrust::device_vector<double> stocks_GPU(stocks);

    // Percorrendo vetor maneira simples
    // for(int i = 0; i < int(stocks_GPU.size()); i++) {
    //     cout << stocks_GPU[i] << endl;
    // }

    cout << "Média nos últimos 10 anos: " << thrust::reduce(stocks_GPU.begin(), stocks_GPU.end(), 0, thrust::plus<double>())/int(stocks_GPU.size()) << endl;
    cout << "Média no último ano: " << thrust::reduce(stocks_GPU.end() - 365, stocks_GPU.end(), 0, thrust::plus<double>())/365 << endl;
    return 0;
}   

Overwriting google_stocks.cu


In [31]:
!nvcc -Wno-deprecated-gpu-targets -arch=sm_37 -std=c++14 -o google_stocks google_stocks.cu

In [32]:
!./google_stocks < stocks.txt

Média nos últimos 10 anos: 493
Média no último ano: 959


#### Exercício Apple Microsoft Stocks

In [44]:
%%writefile apple_microsoft_stocks.cu

#include<iostream>
#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
using namespace std;

#define NStocks 2518

int main() {
    thrust::device_vector<double> appl_stocks(NStocks);
    thrust::device_vector<double> msft_stocks(NStocks);

    for(int i = 0; i < NStocks; i++) {
        double appl_stock, msft_stock;

        cin >> appl_stock >> msft_stock;

        appl_stocks[i] = appl_stock;
        msft_stocks[i] = msft_stock;
    }

    thrust::device_vector<double> stocks_diff(NStocks);
    thrust::transform(appl_stocks.begin(), appl_stocks.end(), msft_stocks.begin(), stocks_diff.begin(), thrust::minus<double>());

    cout << thrust::reduce(stocks_diff.begin(), stocks_diff.end(), 0, thrust::plus<double>())/NStocks << endl;
    return 0;
}   

Overwriting apple_microsoft_stocks.cu


In [45]:
!nvcc -Wno-deprecated-gpu-targets -arch=sm_37 -std=c++14 -o apple_microsoft_stocks apple_microsoft_stocks.cu

In [46]:
!./apple_microsoft_stocks < stocks2.csv

-39
